[toc]

# DL 学习笔记-6-GRU-自定义GRU类

## 实现

### 数据产生

在定义 GRU 之前，我们先明确我们的输入的数据的形状。我们输入的数据是序列数据。序列数据一般有两种组织方式：batch_major 和 time_major。
batch_major 对数据的组织方式是： `[batch_size, n_sequences, n_features]`
time_major 对数据的组织方式是： `[n_sequences, batch_size, n_features]` 。
tensorflow 默认的是 batch_major 的组织方式，因此我们的数据也采用 batch_major 的组织方式

In [1]:
import tensorflow as tf

batch_size = 2
n_sequences = 3
n_features = 5
units = 4 # gru 输出单元的个数

tf.random.set_seed(123)
x_train = tf.random.normal(shape=(batch_size, n_sequences, n_features), dtype=tf.float32)

### 自定义 GRU 类

下面，我们来自使用 Tensorflow 的类来自定义 GRU 类，并和 Tensorflow 原始的 GRU 类的结果进行比较。

$$
\begin{array}{l}
z_{t}=\sigma\left(W_{z} \cdot\left[h_{t-1}, x_{t}\right]\right)  \\
r_{t}=\sigma\left(W_{r} \cdot\left[h_{t-1}, x_{t}\right]\right) \\
\tilde{h}_{t}=\tanh \left(W \cdot\left[r_{t} * h_{t-1}, x_{t}\right]\right) \\
h_{t}=\left(1-z_{t}\right) * h_{t-1}+z_{t} * \tilde{h}_{t}
\end{array}
$$

首先，我们直接利用上面的公式来实现一个 GRU 类。我们的 GRU 类是继承自 `tf.keras.layers.Layer`，注意到上面的公式中省略了 bias 项。因此我们下面的实现中也不设置 bias。但是 tensorflow 自带的 tf.keras.layers.GRU 是添加 bias 和

In [2]:
class customGRU1(tf.keras.layers.Layer):

    def __init__(self, units, name="customGRU", **kwargs):
        super(customGRU1, self).__init__(name=name, **kwargs)
        self.units = units

    def build(self, input_shape):
        input_dim = input_shape[-1] # input_dim 就是 输入的 x 的特征数
        self.Wz = self.add_weight("Wz", shape=[self.units + input_dim, self.units])
        self.Wr = self.add_weight("Wr", shape=[self.units + input_dim, self.units])
        self.W = self.add_weight("W", shape=[self.units + input_dim, self.units])
        super(customGRU1, self).build(input_shape)

    def gru_cell_forward(self, xt, h_prev):
        concat_x = tf.concat([h_prev, xt], axis=1)
        update_gate = tf.nn.sigmoid(tf.matmul(concat_x, self.Wz))
        reset_gate = tf.nn.sigmoid(tf.matmul(concat_x, self.Wr))
        ht_tilde = tf.tanh(tf.matmul(tf.concat([reset_gate * h_prev, xt], axis=1), self.W))
        ht = (1 - update_gate) * h_prev + update_gate * ht_tilde
        return ht

    def call(self, x):
        n_samples, n_sequences = x.shape[0], x.shape[1]
        h_prev = tf.random.normal(shape=(n_samples, self.units))
        for t in range(n_sequences):
            ht = self.gru_cell_forward(x[:, t, :], h_prev)
            h_prev = ht
        return ht

In [3]:
mygru1 = customGRU1(units)
mygru1.build(input_shape=(None, n_sequences, n_features))
mygru1(x_train)

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[-0.3312275 , -0.07250792,  0.34863517,  0.26469705],
       [-0.1867469 ,  0.51856804, -0.45318753, -0.01636913]],
      dtype=float32)>

### 参考 Tensorflow 实现的 GRU 类

上面的 customGRU1 是直接使用公式实现的。既然有 customGRU1，当然有 customGRU2 了！下面的 `customGRU2` 参考了 Tensorflow 的 GRUcell源码实现 [tensorflow/recurrent.py at 2b96f3662bd776e277f86997659e61046b56c315 · tensorflow/tensorflow](https://github.com/tensorflow/tensorflow/blob/2b96f3662bd776e277f86997659e61046b56c315/tensorflow/python/keras/layers/recurrent.py)

源码中，没有直接使用公式。而是对公式进行了一些变换。

首先，在上面的公式中，$W_z$,$W_r$, $W$ 的维度都是 `[units, units + n_features]`。

对于可以将其分别分解为两部分
$W_z = [recurrent_z, x_z]$, $W_r = [recurrent_r, x_r]$, $W = [recurrent_h, x_h]$

其中，shape(x_r) = shape(x_z) = shape(x_h) = [units, n_features]

shape(recurrent_r) = shape(recurrent_z) = shape(recurrent_h) = [units, units]。

然后将 x_z, x_r, x_h 拼接起来，叫做 kernel。将 recurent_r, recurrent_z, recurrent_h 拼接起来，叫做 recurrent_kernel。类比 rnn 中的 kernel 和 recurrent_kernel。

同时，这新拼接起来之后，公式会进行变换。在原来的公式中，x_z, x_r, x_h 都会和 $x_t$ 相乘。当 `x_z`, `x_r`, `x_h` 合并为 kernel 后，可以进行批量操作。 recurrent_kernel 同理，直接看代码比较清楚。

为了控制权重初始化，使用结果可重复，自定义一个初始化权重的函数。

In [4]:
def get_weight(shape, dtype):
    tf.random.set_seed(123)
    return tf.Variable(tf.random.normal(shape=shape), dtype=dtype)

In [5]:
class customGRU2(tf.keras.layers.Layer):

    def __init__(self, units, name="customGRU2", kernel_initializer=None, recurrent_initializer=None, **kwargs):
        super(customGRU2, self).__init__(name=name, **kwargs)
        self.units = units
        self.kernel_initializer = kernel_initializer
        self.recurrent_initializer = recurrent_initializer

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.kernel = self.add_weight("kernel", shape=[input_dim, self.units * 3], initializer=self.kernel_initializer)
        self.recurrent_kernel = self.add_weight("recurrent_kernel", shape=[self.units, self.units * 3], initializer=self.recurrent_initializer)
        super(customGRU2, self).build(input_shape)

    def gru_cell_forward(self, xt, h_prev):
        matrix_x = tf.matmul(xt, self.kernel)
        x_z, x_r, x_h = tf.split(matrix_x, 3, axis=-1)
        matrix_inner = tf.matmul(h_prev, self.recurrent_kernel)
        recurrent_z, recurrent_r, recurrent_h = tf.split(matrix_inner, 3, axis=-1)
        r = tf.nn.sigmoid(x_r + recurrent_r)
        z = tf.nn.sigmoid(x_z + recurrent_z)
        hh = tf.nn.tanh(r * recurrent_h + x_h)
        h = (1 - z) * hh + z * h_prev # 注意，这里与公式有出入
        return h

    def call(self, x):
        n_samples, n_sequences = x.shape[0], x.shape[1]
        h_prev = tf.zeros(shape=(n_samples, self.units))
        for t in range(n_sequences):
            ht = self.gru_cell_forward(x[:, t, :], h_prev)
            h_prev = ht
        return ht

注意，上面实现的最后一步是 h = (1-z) * hh + z * h_prev，因为 tensorflow 的 GRUCell 的源码中是写成这个样子的。
但是这和之前列出的公式有出入，如果是之前列出的公式，应该写成 `h = (1-z) * h_prev + z * hh`。
不过，为了之后和 tensorflow 自带的 GRU 的结果进行比较，我们还是使用了和上述公式相反的 tensorflow 源码中的写法。

但是需要明确的是，这两种写法并无本质差异。

In [6]:
mygru = customGRU2(units, kernel_initializer=get_weight, recurrent_initializer=get_weight)
mygru.build(input_shape=(None, n_features))
mygru(x_train)

# 结果tensorflow自带的 GRU layer
tfgru = tf.keras.layers.GRU(units, use_bias=False, kernel_initializer=get_weight, recurrent_initializer=get_weight)
tfgru.build(input_shape=(None, n_sequences, n_features))
tfgru(x_train)

mygru(x_train) == tfgru(x_train) # 两者结果完全相同

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[-0.83279616,  0.3954636 ,  0.3854519 , -0.2639432 ],
       [-0.49416894,  0.14769015, -0.26631743, -0.02626724]],
      dtype=float32)>

<tf.Tensor: shape=(2, 4), dtype=float32, numpy=
array([[-0.83279616,  0.3954636 ,  0.3854519 , -0.2639432 ],
       [-0.49416894,  0.14769015, -0.26631743, -0.02626724]],
      dtype=float32)>

<tf.Tensor: shape=(2, 4), dtype=bool, numpy=
array([[ True,  True,  True,  True],
       [ True,  True,  True,  True]])>

# References

1. [RNN & LSTM & GRU 的原理与区别 - Jerry_Jin - 博客园](https://www.cnblogs.com/jins-note/p/9715610.html)
2. [tensorflow/recurrent_v2.py at v2.2.0 · tensorflow/tensorflow](https://github.com/tensorflow/tensorflow/blob/v2.2.0/tensorflow/python/keras/layers/recurrent_v2.py#L917-L1207)